In [ ]:
import copy

import numpy as np
import matplotlib.pyplot as pl
import pandas as pd
import pooch
from tqdm.auto import tqdm

from fair import FAIR
from fair.io import read_properties
from fair.interface import initialise, fill

In [ ]:
pd.options.display.max_columns = 50
df_configs = pd.read_csv('../data/fair2.1-parameters/calibrated_constrained_parameters_v1.1.0.csv', index_col=0)
valid_all = df_configs.index
df_configs

In [ ]:
df_solar = pd.read_csv(
    "../data/forcing/solar_erf_timebounds.csv", index_col="year"
)
df_volcanic = pd.read_csv(
    "../data/forcing/volcanic_ERF_monthly_-950001-201912.csv"
)

In [ ]:
volcanic_forcing = np.zeros(352)
for i, year in enumerate(np.arange(1750, 2021)):
    volcanic_forcing[i] = np.mean(
        df_volcanic.loc[
            ((year - 1) <= df_volcanic["year"]) & (df_volcanic["year"] < year)
        ].erf
    )
volcanic_forcing[271:281] = np.linspace(1, 0, 10) * volcanic_forcing[270]

In [ ]:
solar_forcing=np.zeros(352)
solar_forcing_df = pd.read_csv('../data/forcing/solar_erf_timebounds.csv', index_col=0)
solar_forcing[:281] = solar_forcing_df['erf'].values[:281]
solar_forcing[271:281] = np.linspace(1, 0, 10) * solar_forcing[271:281]

In [ ]:
trend_shape = np.ones(352)
trend_shape[:271] = np.linspace(0, 1, 271)

In [ ]:
df_methane = pd.read_csv(
    "../data/fair2.1-parameters/CH4_lifetime.csv",
    index_col=0,
)

In [ ]:
f = FAIR(ch4_method="Thornhill2021")
f.define_time(1750, 2101, 1)
scenarios = ['ssp126']
f.define_scenarios(scenarios)
species, properties = read_properties()
f.define_species(species, properties)
f.define_configs(list(valid_all))
f.allocate()
f.fill_from_rcmip()

In [ ]:
# Fix NOx.
rcmip_emissions_file = pooch.retrieve(
    url="doi:10.5281/zenodo.4589756/rcmip-emissions-annual-means-v5-1-0.csv",
    known_hash="md5:4044106f55ca65b094670e7577eaf9b3",
)
df_emis = pd.read_csv(rcmip_emissions_file)
gfed_sectors = [
    "Emissions|NOx|MAGICC AFOLU|Agricultural Waste Burning",
    "Emissions|NOx|MAGICC AFOLU|Forest Burning",
    "Emissions|NOx|MAGICC AFOLU|Grassland Burning",
    "Emissions|NOx|MAGICC AFOLU|Peat Burning",
]
for scenario in scenarios:
    f.emissions.loc[dict(specie="NOx", scenario=scenario)] = (
        df_emis.loc[
            (df_emis["Scenario"] == scenario)
            & (df_emis["Region"] == "World")
            & (df_emis["Variable"].isin(gfed_sectors)),
            "1750":"2300",
        ]
        .interpolate(axis=1)
        .values.squeeze()
        .sum(axis=0)
        * 46.006
        / 30.006
        + df_emis.loc[
            (df_emis["Scenario"] == scenario)
            & (df_emis["Region"] == "World")
            & (df_emis["Variable"] == "Emissions|NOx|MAGICC AFOLU|Agriculture"),
            "1750":"2300",
        ]
        .interpolate(axis=1)
        .values.squeeze()
        + df_emis.loc[
            (df_emis["Scenario"] == scenario)
            & (df_emis["Region"] == "World")
            & (df_emis["Variable"] == "Emissions|NOx|MAGICC Fossil and Industrial"),
            "1750":"2300",
        ]
        .interpolate(axis=1)
        .values.squeeze()
    )[:351][:, None]

In [ ]:
# solar and volcanic forcing
fill(
    f.forcing,
    volcanic_forcing[:, None, None] * df_configs["scale Volcanic"].values.squeeze(),
    specie="Volcanic",
)
fill(
    f.forcing,
    solar_forcing[:, None, None] * df_configs["solar_amplitude"].values.squeeze()
    + trend_shape[:, None, None] * df_configs["solar_trend"].values.squeeze(),
    specie="Solar",
)

# climate response
fill(f.climate_configs["ocean_heat_capacity"], df_configs.loc[:, "c1":"c3"].values)
fill(
    f.climate_configs["ocean_heat_transfer"],
    df_configs.loc[:, "kappa1":"kappa3"].values,
)
fill(f.climate_configs["deep_ocean_efficacy"], df_configs["epsilon"].values.squeeze())
fill(f.climate_configs["gamma_autocorrelation"], df_configs["gamma"].values.squeeze())
fill(f.climate_configs["sigma_eta"], df_configs["sigma_eta"].values.squeeze())
fill(f.climate_configs["sigma_xi"], df_configs["sigma_xi"].values.squeeze())
fill(f.climate_configs["seed"], df_configs["seed"])
fill(f.climate_configs["stochastic_run"], False)
fill(f.climate_configs["use_seed"], True)
fill(f.climate_configs["forcing_4co2"], df_configs["F_4xCO2"])

# species level
f.fill_species_configs()

# carbon cycle
fill(f.species_configs["iirf_0"], df_configs["r0"].values.squeeze(), specie="CO2")
fill(
    f.species_configs["iirf_airborne"], df_configs["rA"].values.squeeze(), specie="CO2"
)
fill(f.species_configs["iirf_uptake"], df_configs["rU"].values.squeeze(), specie="CO2")
fill(
    f.species_configs["iirf_temperature"],
    df_configs["rT"].values.squeeze(),
    specie="CO2",
)

# aerosol indirect
fill(f.species_configs["aci_scale"], df_configs["beta"].values.squeeze())
fill(
    f.species_configs["aci_shape"],
    df_configs["shape Sulfur"].values.squeeze(),
    specie="Sulfur",
)
fill(
    f.species_configs["aci_shape"], df_configs["shape BC"].values.squeeze(), specie="BC"
)
fill(
    f.species_configs["aci_shape"], df_configs["shape OC"].values.squeeze(), specie="OC"
)

# methane lifetime baseline and sensitivity
fill(
    f.species_configs["unperturbed_lifetime"],
    df_methane.loc["historical_best", "base"],
    specie="CH4",
)
fill(
    f.species_configs["ch4_lifetime_chemical_sensitivity"],
    df_methane.loc["historical_best", "CH4"],
    specie="CH4",
)
fill(
    f.species_configs["ch4_lifetime_chemical_sensitivity"],
    df_methane.loc["historical_best", "N2O"],
    specie="N2O",
)
fill(
    f.species_configs["ch4_lifetime_chemical_sensitivity"],
    df_methane.loc["historical_best", "VOC"],
    specie="VOC",
)
fill(
    f.species_configs["ch4_lifetime_chemical_sensitivity"],
    df_methane.loc["historical_best", "NOx"],
    specie="NOx",
)
fill(
    f.species_configs["ch4_lifetime_chemical_sensitivity"],
    df_methane.loc["historical_best", "HC"],
    specie="Equivalent effective stratospheric chlorine",
)
fill(
    f.species_configs["lifetime_temperature_sensitivity"],
    df_methane.loc["historical_best", "temp"],
)

# emissions adjustments for N2O and CH4 (we don't want to make these defaults as people
# might wanna run pulse expts with these gases)
fill(f.species_configs["baseline_emissions"], 19.019783117809567, specie="CH4")
fill(f.species_configs["baseline_emissions"], 0.08602230754, specie="N2O")
fill(f.species_configs["baseline_emissions"], 19.423526730206152, specie="NOx")

# aerosol direct
for specie in [
    "BC",
    "CH4",
    "N2O",
    "NH3",
    "NOx",
    "OC",
    "Sulfur",
    "VOC",
    "Equivalent effective stratospheric chlorine",
]:
    fill(
        f.species_configs["erfari_radiative_efficiency"],
        df_configs[f"ari {specie}"],
        specie=specie,
    )

# forcing scaling
for specie in [
    "CO2",
    "CH4",
    "N2O",
    "Stratospheric water vapour",
    "Contrails",
    "Light absorbing particles on snow and ice",
    "Land use",
]:
    fill(
        f.species_configs["forcing_scale"],
        df_configs[f"scale {specie}"].values.squeeze(),
        specie=specie,
    )

for specie in [
    "CFC-11",
    "CFC-12",
    "CFC-113",
    "CFC-114",
    "CFC-115",
    "HCFC-22",
    "HCFC-141b",
    "HCFC-142b",
    "CCl4",
    "CHCl3",
    "CH2Cl2",
    "CH3Cl",
    "CH3CCl3",
    "CH3Br",
    "Halon-1211",
    "Halon-1301",
    "Halon-2402",
    "CF4",
    "C2F6",
    "C3F8",
    "c-C4F8",
    "C4F10",
    "C5F12",
    "C6F14",
    "C7F16",
    "C8F18",
    "NF3",
    "SF6",
    "SO2F2",
    "HFC-125",
    "HFC-134a",
    "HFC-143a",
    "HFC-152a",
    "HFC-227ea",
    "HFC-23",
    "HFC-236fa",
    "HFC-245fa",
    "HFC-32",
    "HFC-365mfc",
    "HFC-4310mee",
]:
    fill(
        f.species_configs["forcing_scale"],
        df_configs["scale minorGHG"].values.squeeze(),
        specie=specie,
    )

# ozone
for specie in [
    "CH4",
    "N2O",
    "Equivalent effective stratospheric chlorine",
    "CO",
    "VOC",
    "NOx",
]:
    fill(
        f.species_configs["ozone_radiative_efficiency"],
        df_configs[f"o3 {specie}"],
        specie=specie,
    )

# tune down volcanic efficacy
fill(f.species_configs["forcing_efficacy"], 0.6, specie="Volcanic")


# initial condition of CO2 concentration (but not baseline for forcing calculations)
fill(
    f.species_configs["baseline_concentration"],
    df_configs["co2_concentration_1750"].values.squeeze(),
    specie="CO2",
)

initialise(f.concentration, f.species_configs['baseline_concentration'])
initialise(f.forcing, 0)
initialise(f.temperature, 0)
initialise(f.airborne_emissions, 0)
initialise(f.cumulative_emissions, 0)

f.run()

In [ ]:
f_fut = {}

In [ ]:
f_fut[2021] = FAIR(ch4_method="Thornhill2021")

In [ ]:
f_fut[2021].define_time(1750, 2022, 1)

In [ ]:
f_fut[2021].define_scenarios(scenarios)

In [ ]:
species, properties = read_properties()

In [ ]:
f_fut[2021].define_species(species, properties)

In [ ]:
f_fut[2021].define_configs([valid_all[10]])

In [ ]:
f_fut[2021].allocate()

In [ ]:
f_fut[2021].fill_from_rcmip()

In [ ]:
for scenario in scenarios:
    f_fut[2021].emissions.loc[dict(specie="NOx", scenario=scenario)] = (
        df_emis.loc[
            (df_emis["Scenario"] == scenario)
            & (df_emis["Region"] == "World")
            & (df_emis["Variable"].isin(gfed_sectors)),
            "1750":"2300",
        ]
        .interpolate(axis=1)
        .values.squeeze()
        .sum(axis=0)
        * 46.006
        / 30.006
        + df_emis.loc[
            (df_emis["Scenario"] == scenario)
            & (df_emis["Region"] == "World")
            & (df_emis["Variable"] == "Emissions|NOx|MAGICC AFOLU|Agriculture"),
            "1750":"2300",
        ]
        .interpolate(axis=1)
        .values.squeeze()
        + df_emis.loc[
            (df_emis["Scenario"] == scenario)
            & (df_emis["Region"] == "World")
            & (df_emis["Variable"] == "Emissions|NOx|MAGICC Fossil and Industrial"),
            "1750":"2300",
        ]
        .interpolate(axis=1)
        .values.squeeze()
    )[:272][:, None]

In [ ]:
# solar and volcanic forcing
fill(
    f_fut[2021].forcing,
    volcanic_forcing[:273, None, None] * df_configs.loc[valid_all[10], "scale Volcanic"],
    specie="Volcanic",
)
fill(
    f_fut[2021].forcing,
    solar_forcing[:273, None, None] * df_configs.loc[valid_all[10], "solar_amplitude"]
    + trend_shape[:273, None, None] * df_configs.loc[valid_all[10], "solar_trend"],
    specie="Solar",
)

# climate response
fill(f_fut[2021].climate_configs["ocean_heat_capacity"], df_configs.loc[valid_all[10], "c1":"c3"].values)
fill(
    f_fut[2021].climate_configs["ocean_heat_transfer"],
    df_configs.loc[valid_all[10], "kappa1":"kappa3"].values,
)
fill(f_fut[2021].climate_configs["deep_ocean_efficacy"], df_configs.loc[valid_all[10], "epsilon"])
fill(f_fut[2021].climate_configs["gamma_autocorrelation"], df_configs.loc[valid_all[10], "gamma"])
fill(f_fut[2021].climate_configs["sigma_eta"], df_configs.loc[valid_all[10], "sigma_eta"])
fill(f_fut[2021].climate_configs["sigma_xi"], df_configs.loc[valid_all[10], "sigma_xi"])
fill(f_fut[2021].climate_configs["seed"], df_configs.loc[valid_all[10], "seed"])
fill(f_fut[2021].climate_configs["stochastic_run"], False)
fill(f_fut[2021].climate_configs["use_seed"], True)
fill(f_fut[2021].climate_configs["forcing_4co2"], df_configs.loc[valid_all[10], "F_4xCO2"])

# species level
f_fut[2021].fill_species_configs()

# carbon cycle
fill(f_fut[2021].species_configs["iirf_0"], df_configs.loc[valid_all[10],"r0"], specie="CO2")
fill(
    f_fut[2021].species_configs["iirf_airborne"], df_configs.loc[valid_all[10],"rA"], specie="CO2"
)
fill(f_fut[2021].species_configs["iirf_uptake"], df_configs.loc[valid_all[10],"rU"], specie="CO2")
fill(
    f_fut[2021].species_configs["iirf_temperature"],
    df_configs.loc[valid_all[10],"rT"],
    specie="CO2",
)

# aerosol indirect
fill(f_fut[2021].species_configs["aci_scale"], df_configs.loc[valid_all[10],"beta"])
fill(
    f_fut[2021].species_configs["aci_shape"],
    df_configs.loc[valid_all[10], "shape Sulfur"],
    specie="Sulfur",
)
fill(
    f_fut[2021].species_configs["aci_shape"], df_configs.loc[valid_all[10], "shape BC"], specie="BC"
)
fill(
    f_fut[2021].species_configs["aci_shape"], df_configs.loc[valid_all[10], "shape OC"], specie="OC"
)

# methane lifetime baseline and sensitivity
fill(
    f_fut[2021].species_configs["unperturbed_lifetime"],
    df_methane.loc["historical_best", "base"],
    specie="CH4",
)
fill(
    f_fut[2021].species_configs["ch4_lifetime_chemical_sensitivity"],
    df_methane.loc["historical_best", "CH4"],
    specie="CH4",
)
fill(
    f_fut[2021].species_configs["ch4_lifetime_chemical_sensitivity"],
    df_methane.loc["historical_best", "N2O"],
    specie="N2O",
)
fill(
    f_fut[2021].species_configs["ch4_lifetime_chemical_sensitivity"],
    df_methane.loc["historical_best", "VOC"],
    specie="VOC",
)
fill(
    f_fut[2021].species_configs["ch4_lifetime_chemical_sensitivity"],
    df_methane.loc["historical_best", "NOx"],
    specie="NOx",
)
fill(
    f_fut[2021].species_configs["ch4_lifetime_chemical_sensitivity"],
    df_methane.loc["historical_best", "HC"],
    specie="Equivalent effective stratospheric chlorine",
)
fill(
    f_fut[2021].species_configs["lifetime_temperature_sensitivity"],
    df_methane.loc["historical_best", "temp"],
)

# emissions adjustments for N2O and CH4 (we don't want to make these defaults as people
# might wanna run pulse expts with these gases)
fill(f_fut[2021].species_configs["baseline_emissions"], 19.019783117809567, specie="CH4")
fill(f_fut[2021].species_configs["baseline_emissions"], 0.08602230754, specie="N2O")
fill(f_fut[2021].species_configs["baseline_emissions"], 19.423526730206152, specie="NOx")

# aerosol direct
for specie in [
    "BC",
    "CH4",
    "N2O",
    "NH3",
    "NOx",
    "OC",
    "Sulfur",
    "VOC",
    "Equivalent effective stratospheric chlorine",
]:
    fill(
        f_fut[2021].species_configs["erfari_radiative_efficiency"],
        df_configs.loc[valid_all[10], f"ari {specie}"],
        specie=specie,
    )

# forcing scaling
for specie in [
    "CO2",
    "CH4",
    "N2O",
    "Stratospheric water vapour",
    "Contrails",
    "Light absorbing particles on snow and ice",
    "Land use",
]:
    fill(
        f_fut[2021].species_configs["forcing_scale"],
        df_configs.loc[valid_all[10], f"scale {specie}"],
        specie=specie,
    )

for specie in [
    "CFC-11",
    "CFC-12",
    "CFC-113",
    "CFC-114",
    "CFC-115",
    "HCFC-22",
    "HCFC-141b",
    "HCFC-142b",
    "CCl4",
    "CHCl3",
    "CH2Cl2",
    "CH3Cl",
    "CH3CCl3",
    "CH3Br",
    "Halon-1211",
    "Halon-1301",
    "Halon-2402",
    "CF4",
    "C2F6",
    "C3F8",
    "c-C4F8",
    "C4F10",
    "C5F12",
    "C6F14",
    "C7F16",
    "C8F18",
    "NF3",
    "SF6",
    "SO2F2",
    "HFC-125",
    "HFC-134a",
    "HFC-143a",
    "HFC-152a",
    "HFC-227ea",
    "HFC-23",
    "HFC-236fa",
    "HFC-245fa",
    "HFC-32",
    "HFC-365mfc",
    "HFC-4310mee",
]:
    fill(
        f_fut[2021].species_configs["forcing_scale"],
        df_configs.loc[valid_all[10],"scale minorGHG"],
        specie=specie,
    )

# ozone
for specie in [
    "CH4",
    "N2O",
    "Equivalent effective stratospheric chlorine",
    "CO",
    "VOC",
    "NOx",
]:
    fill(
        f_fut[2021].species_configs["ozone_radiative_efficiency"],
        df_configs.loc[valid_all[10], f"o3 {specie}"],
        specie=specie,
    )

# tune down volcanic efficacy
fill(f_fut[2021].species_configs["forcing_efficacy"], 0.6, specie="Volcanic")


# initial condition of CO2 concentration (but not baseline for forcing calculations)
fill(
    f_fut[2021].species_configs["baseline_concentration"],
    df_configs.loc[valid_all[10],"co2_concentration_1750"],
    specie="CO2",
)

initialise(f_fut[2021].concentration, f_fut[2021].species_configs['baseline_concentration'])
initialise(f_fut[2021].forcing, 0)
initialise(f_fut[2021].temperature, 0)
initialise(f_fut[2021].airborne_emissions, 0)
initialise(f_fut[2021].cumulative_emissions, 0)

f_fut[2021].run()

In [ ]:
f_fut[2021].species_configs['forcing_scale']

In [ ]:
fill(f_fut[2021].concentration, np.nan)
initialise(f_fut[2021].concentration, f_fut[2021].species_configs['baseline_concentration'])
initialise(f_fut[2021].forcing, 0)
initialise(f_fut[2021].temperature, 0)
initialise(f_fut[2021].airborne_emissions, 0)
initialise(f_fut[2021].cumulative_emissions, 0)

In [ ]:
f_fut[2021].run()

In [ ]:
for year in tqdm(range(2022, 2101)):
    f_fut[year] = FAIR(ch4_method="Thornhill2021")
    f_fut[year].define_time(year, year+1, 1)
    f_fut[year].define_scenarios(scenarios)
    f_fut[year].define_species(species, properties)
    f_fut[year].define_configs([valid_all[10]])
    f_fut[year].allocate()
    f_fut[year].fill_from_rcmip()
    f_fut[year].fill_species_configs()
    
    for scenario in scenarios:
        f_fut[year].emissions.loc[dict(specie="NOx", scenario=scenario)] = (
            df_emis.loc[
                (df_emis["Scenario"] == scenario)
                & (df_emis["Region"] == "World")
                & (df_emis["Variable"].isin(gfed_sectors)),
                "1750":"2300",
            ]
            .interpolate(axis=1)
            .values.squeeze()
            .sum(axis=0)
            * 46.006
            / 30.006
            + df_emis.loc[
                (df_emis["Scenario"] == scenario)
                & (df_emis["Region"] == "World")
                & (df_emis["Variable"] == "Emissions|NOx|MAGICC AFOLU|Agriculture"),
                "1750":"2300",
            ]
            .interpolate(axis=1)
            .values.squeeze()
            + df_emis.loc[
                (df_emis["Scenario"] == scenario)
                & (df_emis["Region"] == "World")
                & (df_emis["Variable"] == "Emissions|NOx|MAGICC Fossil and Industrial"),
                "1750":"2300",
            ]
            .interpolate(axis=1)
            .values.squeeze()
        )[year-1750:year-1749][:, None]
    
    # climate response
    fill(f_fut[year].climate_configs["ocean_heat_capacity"], df_configs.loc[valid_all[10], "c1":"c3"].values)
    fill(
        f_fut[year].climate_configs["ocean_heat_transfer"],
        df_configs.loc[valid_all[10], "kappa1":"kappa3"].values,
    )
    fill(f_fut[year].climate_configs["deep_ocean_efficacy"], df_configs.loc[valid_all[10], "epsilon"])
    fill(f_fut[year].climate_configs["gamma_autocorrelation"], df_configs.loc[valid_all[10], "gamma"])
    fill(f_fut[year].climate_configs["sigma_eta"], df_configs.loc[valid_all[10], "sigma_eta"])
    fill(f_fut[year].climate_configs["sigma_xi"], df_configs.loc[valid_all[10], "sigma_xi"])
    fill(f_fut[year].climate_configs["seed"], df_configs.loc[valid_all[10], "seed"])
    fill(f_fut[year].climate_configs["stochastic_run"], False)
    fill(f_fut[year].climate_configs["use_seed"], True)
    fill(f_fut[year].climate_configs["forcing_4co2"], df_configs.loc[valid_all[10], "F_4xCO2"])

    # species level
    f_fut[year].fill_species_configs()

    # carbon cycle
    fill(f_fut[year].species_configs["iirf_0"], df_configs.loc[valid_all[10],"r0"], specie="CO2")
    fill(
        f_fut[year].species_configs["iirf_airborne"], df_configs.loc[valid_all[10],"rA"], specie="CO2"
    )
    fill(f_fut[year].species_configs["iirf_uptake"], df_configs.loc[valid_all[10],"rU"], specie="CO2")
    fill(
        f_fut[year].species_configs["iirf_temperature"],
        df_configs.loc[valid_all[10],"rT"],
        specie="CO2",
    )

    # aerosol indirect
    fill(f_fut[year].species_configs["aci_scale"], df_configs.loc[valid_all[10],"beta"])
    fill(
        f_fut[year].species_configs["aci_shape"],
        df_configs.loc[valid_all[10], "shape Sulfur"],
        specie="Sulfur",
    )
    fill(
        f_fut[year].species_configs["aci_shape"], df_configs.loc[valid_all[10], "shape BC"], specie="BC"
    )
    fill(
        f_fut[year].species_configs["aci_shape"], df_configs.loc[valid_all[10], "shape OC"], specie="OC"
    )

    # methane lifetime baseline and sensitivity
    fill(
        f_fut[year].species_configs["unperturbed_lifetime"],
        df_methane.loc["historical_best", "base"],
        specie="CH4",
    )
    fill(
        f_fut[year].species_configs["ch4_lifetime_chemical_sensitivity"],
        df_methane.loc["historical_best", "CH4"],
        specie="CH4",
    )
    fill(
        f_fut[year].species_configs["ch4_lifetime_chemical_sensitivity"],
        df_methane.loc["historical_best", "N2O"],
        specie="N2O",
    )
    fill(
        f_fut[year].species_configs["ch4_lifetime_chemical_sensitivity"],
        df_methane.loc["historical_best", "VOC"],
        specie="VOC",
    )
    fill(
        f_fut[year].species_configs["ch4_lifetime_chemical_sensitivity"],
        df_methane.loc["historical_best", "NOx"],
        specie="NOx",
    )
    fill(
        f_fut[year].species_configs["ch4_lifetime_chemical_sensitivity"],
        df_methane.loc["historical_best", "HC"],
        specie="Equivalent effective stratospheric chlorine",
    )
    fill(
        f_fut[year].species_configs["lifetime_temperature_sensitivity"],
        df_methane.loc["historical_best", "temp"],
    )

    # emissions adjustments for N2O and CH4 (we don't want to make these defaults as people
    # might wanna run pulse expts with these gases)
    fill(f_fut[year].species_configs["baseline_emissions"], 19.019783117809567, specie="CH4")
    fill(f_fut[year].species_configs["baseline_emissions"], 0.08602230754, specie="N2O")
    fill(f_fut[year].species_configs["baseline_emissions"], 19.423526730206152, specie="NOx")

    # aerosol direct
    for specie in [
        "BC",
        "CH4",
        "N2O",
        "NH3",
        "NOx",
        "OC",
        "Sulfur",
        "VOC",
        "Equivalent effective stratospheric chlorine",
    ]:
        fill(
            f_fut[2021].species_configs["erfari_radiative_efficiency"],
            df_configs.loc[valid_all[10], f"ari {specie}"],
            specie=specie,
        )

    # forcing scaling
    for specie in [
        "CO2",
        "CH4",
        "N2O",
        "Stratospheric water vapour",
        "Contrails",
        "Light absorbing particles on snow and ice",
        "Land use",
    ]:
        fill(
            f_fut[2021].species_configs["forcing_scale"],
            df_configs.loc[valid_all[10], f"scale {specie}"],
            specie=specie,
        )

    for specie in [
        "CFC-11",
        "CFC-12",
        "CFC-113",
        "CFC-114",
        "CFC-115",
        "HCFC-22",
        "HCFC-141b",
        "HCFC-142b",
        "CCl4",
        "CHCl3",
        "CH2Cl2",
        "CH3Cl",
        "CH3CCl3",
        "CH3Br",
        "Halon-1211",
        "Halon-1301",
        "Halon-2402",
        "CF4",
        "C2F6",
        "C3F8",
        "c-C4F8",
        "C4F10",
        "C5F12",
        "C6F14",
        "C7F16",
        "C8F18",
        "NF3",
        "SF6",
        "SO2F2",
        "HFC-125",
        "HFC-134a",
        "HFC-143a",
        "HFC-152a",
        "HFC-227ea",
        "HFC-23",
        "HFC-236fa",
        "HFC-245fa",
        "HFC-32",
        "HFC-365mfc",
        "HFC-4310mee",
    ]:
        fill(
            f_fut[2021].species_configs["forcing_scale"],
            df_configs.loc[valid_all[10],"scale minorGHG"],
            specie=specie,
        )

    # ozone
    for specie in [
        "CH4",
        "N2O",
        "Equivalent effective stratospheric chlorine",
        "CO",
        "VOC",
        "NOx",
    ]:
        fill(
            f_fut[year].species_configs["ozone_radiative_efficiency"],
            df_configs.loc[valid_all[10], f"o3 {specie}"],
            specie=specie,
        )

    # tune down volcanic efficacy
    fill(f_fut[year].species_configs["forcing_efficacy"], 0.6, specie="Volcanic")


    # initial condition of CO2 concentration (but not baseline for forcing calculations)
    fill(
        f_fut[year].species_configs["baseline_concentration"],
        df_configs.loc[valid_all[10],"co2_concentration_1750"],
        specie="CO2",
    )
    
    while True:
        initialise(f_fut[year].concentration, f_fut[year-1].concentration[-1, ...])
        initialise(f_fut[year].forcing, f_fut[year-1].forcing[-1, ...])
        initialise(f_fut[year].temperature, f_fut[year-1].temperature[-1, ...])
        initialise(f_fut[year].airborne_emissions, f_fut[year-1].airborne_emissions[-1, ...])
        initialise(f_fut[year].cumulative_emissions, f_fut[year-1].cumulative_emissions[-1, ...])
        initialise(f_fut[year].alpha_lifetime, f_fut[year-1].alpha_lifetime[-1, ...])
        f_fut[year].gas_partitions=copy.deepcopy(f_fut[year-1].gas_partitions)
        
        # Volcanic forcing
        fill(f_fut[year].forcing, volcanic_forcing[year-1750:year-1749, None, None] * df_configs.loc[valid_all[10], 'scale Volcanic'], specie='Volcanic')

        # Solar forcing
        fill(f_fut[year].forcing, 
             solar_forcing[year-1750:year-1749, None, None] * 
             df_configs.loc[valid_all[10], 'solar_amplitude'] + 
             trend_shape[year-1750:year-1749, None, None] * df_configs.loc[valid_all[10], 'solar_trend'],
             specie='Solar'
        )

        # do the run
        f_fut[year].run(progress=False)

        # naive optimizer!
        t_anom = (f_fut[year].temperature[-1, 0, 0, 0] - f_fut[2021].temperature[100:151, 0, 0, 0].mean(axis=0))
        if t_anom > 1.5:
            print(f"temperature in {year} is {t_anom}. Reducing methane emissions by 10 MtCH4 to {f_fut[year].emissions[0, 0, 0, 3] - 10}")
            f_fut[year].emissions[0, 0, 0, 3] = f_fut[year].emissions[0,0,0,3] - 10
        else:
            break

In [ ]:
for year in range(2021, 2100):
    pl.plot(f_fut[year].timebounds, f_fut[year].temperature[:, 0, 0, 0], color='k')
pl.plot(f.timebounds, f.temperature[:,0,10,0], color='r')

In [ ]:
emissions_opt = np.zeros(350)
emissions_opt[:272] = f_fut[2021].emissions[:, 0, 0, 3]
for year in range(2022, 2100):
    emissions_opt[year-1750] = f_fut[year].emissions[:, 0, 0, 3]
    
concentration_opt = np.zeros(351)
concentration_opt[:273] = f_fut[2021].concentration[:, 0, 0, 3]
for year in range(2022, 2100):
    concentration_opt[year-1749] = f_fut[year].concentration[1, 0, 0, 3]

forcing_opt = np.zeros(351)
forcing_opt[:273] = f_fut[2021].forcing[:, 0, 0, 3]
for year in range(2022, 2100):
    forcing_opt[year-1749] = f_fut[year].forcing[1, 0, 0, 3]
    
forcing_sum_opt = np.zeros(351)
forcing_sum_opt[:273] = f_fut[2021].forcing_sum[:, 0, 0]
for year in range(2022, 2100):
    forcing_sum_opt[year-1749] = f_fut[year].forcing_sum[1, 0, 0]
    
temperature_opt = np.zeros(351)
temperature_opt[:273] = f_fut[2021].temperature[:, 0, 0, 0]
for year in range(2022, 2100):
    temperature_opt[year-1749] = f_fut[year].temperature[1, 0, 0, 0]
    
lifetime_opt = np.zeros(351)
lifetime_opt[:273] = f_fut[2021].alpha_lifetime[:, 0, 0, 3] * df_methane.loc["historical_best", "base"]
for year in range(2022, 2100):
    lifetime_opt[year-1749] = f_fut[year].alpha_lifetime[0, 0, 0, 3] * df_methane.loc["historical_best", "base"]

In [ ]:
fig, ax = pl.subplots(1, 2, squeeze=True)
ax[0].plot(f.timepoints, f.emissions[:, 0, :, 3]);
ax[0].plot(f.timepoints[:-1], emissions_opt);
#ax[0].plot(g.timepoints, g.emissions[:, 0, :, 3]);
ax[0].axhline(0, color='k', lw=0.5, ls=':')
ax[0].set_xlim(1850, 2100)
ax[0].set_ylim(-400, 400)
ax[0].set_title("Net CH$_4$ emissions")
ax[0].set_ylabel("MtCH$_4$ yr$^{-1}$")

ax[1].plot(f.timebounds, f.temperature[:, 0, 10, 0] - f.temperature[100:151, 0, 10, 0].mean(axis=0), label='SSP1-2.6');
ax[1].plot(f.timebounds[:-1], temperature_opt - temperature_opt[100:151].mean(), label='Methane removal');
#ax[1].plot(g.timebounds, g.temperature[:, 0, :, 0] - g.temperature[100:151, 0, :, 0].mean(axis=0), label='Methane removal');
ax[1].axhline(1.5, color='r', lw=0.5, ls=':')
ax[1].set_xlim(1850, 2100)
ax[1].set_ylim(-.5, 2.5)
ax[1].set_title("Global mean temperature anomaly")
ax[1].set_ylabel("K")
ax[1].legend()

#fig.tight_layout()

In [ ]:
np.sum(emissions_opt - f.emissions[:-1, 0, 0, 3])

In [ ]:
fig, ax = pl.subplots(1, 2, squeeze=True)
ax[0].plot(f.timebounds, f.concentration[:, 0, 10, 3]);
ax[0].plot(f.timebounds[:-1], concentration_opt);
#ax[0].plot(g.timepoints, g.emissions[:, 0, :, 3]);
ax[0].axhline(0, color='k', lw=0.5, ls=':')
ax[0].set_xlim(1850, 2100)
ax[0].set_ylim(0, 2000)
ax[0].set_title("CH$_4$ concentration")
ax[0].set_ylabel("ppb")

ax[1].plot(f.timebounds, f.alpha_lifetime[:, 0, 10, 3] * df_methane.loc["historical_best", "base"], label='SSP1-2.6');
ax[1].plot(f.timebounds[:-1], lifetime_opt, label='Methane removal');
#ax[1].plot(g.timebounds, g.temperature[:, 0, :, 0] - g.temperature[100:151, 0, :, 0].mean(axis=0), label='Methane removal');
ax[1].axhline(1.5, color='r', lw=0.5, ls=':')
ax[1].set_xlim(1850, 2100)
ax[1].set_ylim(6, 12)
ax[1].set_title("Methane lifetime")
ax[1].set_ylabel("yr")
ax[1].legend()

fig.tight_layout()

In [ ]:
fig, ax = pl.subplots(1, 2, squeeze=True)
ax[0].plot(f.timebounds, f.forcing[:, 0, 10, 3]);
ax[0].plot(f.timebounds[:-1], forcing_opt);
#ax[0].plot(g.timepoints, g.emissions[:, 0, :, 3]);
ax[0].axhline(0, color='k', lw=0.5, ls=':')
ax[0].set_xlim(1850, 2100)
#ax[0].set_ylim(0, 2000)
ax[0].set_title("CH$_4$ forcing")
ax[0].set_ylabel("W m$^{-2}$")

ax[1].plot(f.timebounds, f.forcing_sum[:, 0, 10], label='SSP1-2.6');
ax[1].plot(f.timebounds[:-1], forcing_sum_opt, label='Methane removal');
#ax[1].plot(g.timebounds, g.temperature[:, 0, :, 0] - g.temperature[100:151, 0, :, 0].mean(axis=0), label='Methane removal');
ax[1].set_xlim(1850, 2100)
ax[1].set_ylim(-2, 4)
ax[1].set_title("Total forcing")
ax[1].set_ylabel("W m$^{-2}$")
ax[1].legend()

fig.tight_layout()